In [79]:
from yummly import Client
import json
import requests
import pandas as pd
import numpy as np
import re

In [80]:
TIMEOUT = 5.0
RETRIES = 0
headers= {'X-Yummly-App-ID':'79663a75', 'X-Yummly-App-Key':'02b233108f476f3110e0f65437c4d6dd'}
# #client = Client(api_id='79663a75', api_key='02b233108f476f3110e0f65437c4d6dd', timeout=TIMEOUT, retries=RETRIES)

# #search = client.search('green eggs and ham')
# match = search.matches[0]

# recipe = client.recipe(match.id)



In [82]:
headers= {'X-Yummly-App-ID':'79663a75', 'X-Yummly-App-Key':'02b233108f476f3110e0f65437c4d6dd'}
url='http://api.yummly.com/v1/api/recipes?'
parameters={'q': 'chicken',
            'maxResult': 500}

response=requests.get(url,headers=headers, params=parameters)

In [83]:
response.status_code

200

In [84]:
chicken=response.json()
type(chicken)


dict

In [85]:
chicken.keys()

[u'matches', u'totalMatchCount', u'attribution', u'facetCounts', u'criteria']

In [86]:
chicken['matches'][0]

{u'attributes': {u'course': [u'Main Dishes'], u'cuisine': [u'Kid-Friendly']},
 u'flavors': {u'bitter': 0.6666666666666666,
  u'meaty': 0.3333333333333333,
  u'piquant': 0.16666666666666666,
  u'salty': 0.8333333333333334,
  u'sour': 0.16666666666666666,
  u'sweet': 0.3333333333333333},
 u'id': u'Chicken-Diavolo-1710459',
 u'imageUrlsBySize': {u'90': u'https://lh3.googleusercontent.com/9463IRbtZXq5ASg1YAlnzH2Zdus_mH9rFPLbEflkZ4GGUT4WJxf2P07zAs4rYyAATViqsnzHRKaHv6YzTh6t=s90-c'},
 u'ingredients': [u'chicken breasts',
  u'italian salad dressing',
  u'spaghetti',
  u'marinara sauce',
  u'red pepper flakes'],
 u'rating': 4,
 u'recipeName': u'Chicken Diavolo',
 u'smallImageUrls': [u'https://lh3.googleusercontent.com/hlJ6ah9T116_R1d5x3qXPSwSaD7jJ7LB_OOJ_JKJGxsbpO1z-F7aI7-sMZyTBI71ZPlbvjqYDct8eD2zWgST=s90'],
 u'sourceDisplayName': u'Cupcake Diaries',
 u'totalTimeInSeconds': 1800}

In [87]:
# all column names
chicken_matches = chicken['matches']

main=['id','rating','recipeName','sourceDisplayName','totalTimeInSeconds']
ingredients=['id', 'ingredients']
attributes=['id', 'attributes']



In [88]:
#x[0]-dictionary of one recipe
main_list = []
ingredients_list = []
attributes_list = []


for food in chicken_matches:

    _d1 = {}
    _d1['id'] = food['id']
    _d1['rating'] = food['rating']
    _d1['recipeName'] = food['recipeName']
    _d1['sourceDisplayName'] = food['sourceDisplayName']
    _d1['totalTimeInSeconds'] = food['totalTimeInSeconds'] 
    main_list.append(_d1)
    
    _d2 = {}
    _d2['id'] =food['id']
    for i in food['ingredients']:
        i = i.lower() # additional code to conver to lowercase
        i = re.sub(r'\d+%\s', '', i) # additional code to remove 1%, 2%, etc
        i = re.sub(r'\xae', '', i) # remove '\xae' characters
        _d2[i] = 1
    ingredients_list.append(_d2)

    _d3 = {}
    _d3['_id'] = food['id']
    for k, v in food['attributes'].items():
        for i in v:
            _d3[i] = 1
    attributes_list.append(_d3)
    
    
from pprint import pprint
pprint(main_list[0])
pprint(ingredients_list[0])
pprint(attributes_list[4])

{'id': u'Chicken-Diavolo-1710459',
 'rating': 4,
 'recipeName': u'Chicken Diavolo',
 'sourceDisplayName': u'Cupcake Diaries',
 'totalTimeInSeconds': 1800}
{u'chicken breasts': 1,
 'id': u'Chicken-Diavolo-1710459',
 u'italian salad dressing': 1,
 u'marinara sauce': 1,
 u'red pepper flakes': 1,
 u'spaghetti': 1}
{u'Kid-Friendly': 1, '_id': u'Parmesan-Chicken-Mini-Pies-685394'}


In [41]:
# read in dictionary for course and cuisine list
course_df = pd.read_csv('course_headers.csv', names=['course'])
cuisine_df = pd.read_csv('cuisine_headers.csv', names=['cuisine'])

In [78]:
course_list = course_df.course
cuisine_list= cuisine_df.cuisine
cuisine_list

0                 American
1             Kid-Friendly
2                  Italian
3                    Asian
4                  Mexican
5     Southern & Soul Food
6                   French
7             Southwestern
8                 Barbecue
9                   Indian
10                 Chinese
11          Cajun & Creole
12           Mediterranean
13                   Greek
14                 English
15                 Spanish
16                    Thai
17                  German
18                Moroccan
19                   Irish
20                Japanese
21                   Cuban
22                Hawaiian
23                 Swedish
24               Hungarian
25              Portuguese
Name: cuisine, dtype: object

In [57]:
#create dictionary for course and cuisine
course_dict={}
cuisine_dict={}
for food in chicken['matches']:
    course_dict[food.get('id')]= food['attributes'].get('course')
    cuisine_dict[food.get('id')]= food['attributes'].get('cuisine')
   

In [47]:
_courses= {}

for k, v in course_dict.iteritems():
    course_val = {}
    for course in course_list:
        try:
            if course in v :
                course_val[course] = 1
            else:
                course_val[course] = 0
        except TypeError:
            course_val[course] = 0
    
        _courses[k] = course_val
        
_cuisines= {}       

for k, v in cuisine_dict.iteritems():
    cuisine_val = {}
    for course in cuisine_list:
        try:
            if course in v :
                cuisine_val[course] = 1
            else:
                cuisine_val[course] = 0
        except TypeError:
            cuisine_val[course] = 0
    
        _cuisines[k] = cuisine_val


In [58]:
_cuisines

{u'15-Minute-Healthy-Roasted-Chicken-and-Veggies-1702651': {'American': 0,
  'Asian': 0,
  'Barbecue': 0,
  'Cajun & Creole': 0,
  'Chinese': 0,
  'Cuban': 0,
  'English': 0,
  'French': 0,
  'German': 0,
  'Greek': 0,
  'Hawaiian': 0,
  'Hungarian': 0,
  'Indian': 0,
  'Irish': 0,
  'Italian': 0,
  'Japanese': 0,
  'Kid-Friendly': 0,
  'Mediterranean': 0,
  'Mexican': 0,
  'Moroccan': 0,
  'Portuguese': 0,
  'Southern & Soul Food': 0,
  'Southwestern': 0,
  'Spanish': 0,
  'Swedish': 0,
  'Thai': 0},
 u'15-Minute-Pesto-Chicken-Breast-1704385': {'American': 0,
  'Asian': 0,
  'Barbecue': 0,
  'Cajun & Creole': 0,
  'Chinese': 0,
  'Cuban': 0,
  'English': 0,
  'French': 0,
  'German': 0,
  'Greek': 0,
  'Hawaiian': 0,
  'Hungarian': 0,
  'Indian': 0,
  'Irish': 0,
  'Italian': 0,
  'Japanese': 0,
  'Kid-Friendly': 0,
  'Mediterranean': 0,
  'Mexican': 0,
  'Moroccan': 0,
  'Portuguese': 0,
  'Southern & Soul Food': 0,
  'Southwestern': 0,
  'Spanish': 0,
  'Swedish': 0,
  'Thai': 0},
 

In [59]:
#create dataframes and save as csv
df_main =pd.DataFrame(main_list)
df_main.to_csv('main.csv', encoding='utf-8')
df_ingredients =pd.DataFrame(ingredients_list)
df_ingredients.to_csv('ingredients.csv', encoding ='utf-8')
df_attributes =pd.DataFrame(attributes_list)
df_attributes.to_csv('_attributes.csv')
df_courses=pd.DataFrame(_courses).transpose()
df_cuisines=pd.DataFrame(_cuisines).transpose()

In [31]:
df_main.head(3)

,id,rating,recipeName,sourceDisplayName,totalTimeInSeconds
0,Pizza-Baked-Chicken-1706825,4,Pizza Baked Chicken,Get In My Belly,3000
1,Parmesan-Chicken-Mini-Pies-685394,4,Parmesan Chicken Mini Pies,None,1500
2,Chicken-and-Avocado-Burritos-1704783,4,Chicken and Avocado Burritos,Closet Cooking,1200


In [32]:
df_ingredients.head(3)

,Chinese rice vinegar,Pillsbury™ Refrigerated Crescent Dinner Rolls,Progresso™ Chicken Broth,Tyson® frozen popcorn chicken pieces,all-purpose flour,allspice,apple cider vinegar,avocado,bacon,barbecue sauce,...,tomato sauce,tomatoes,tortillas,unsalted butter,vegetable oil,water,white vinegar,wine,yellow summer squash,zucchini
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df_attributes.head(3)

,American,Appetizers,Asian,Barbecue,Cajun & Creole,Chinese,Condiments and Sauces,Cuban,Game Day,Greek,...,Snacks,Soups,Southern & Soul Food,Southwestern,Spanish,Summer,Sunday Lunch,Thai,Thanksgiving,_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pizza-Baked-Chicken-1706825
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parmesan-Chicken-Mini-Pies-685394
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,Chicken-and-Avocado-Burritos-1704783


In [63]:
df_courses.head(3)
df_courses.shape


(500, 13)

In [64]:
df_cuisines.head(3)
df_cuisines.shape

(500, 26)